In [1]:
#from sunpy.net import hek
#from sunpy.net import hek2vso
import matplotlib.pyplot as plt
%matplotlib inline
#matplotlib qt
import shapely
from shapely.wkt import dumps, loads
import numpy as np
import pandas as pd
from shapely.geometry.polygon import LinearRing, Polygon
from datetime import timedelta
import datetime
#from sunpy.time import *
from dateutil.parser import parse
import matplotlib.cm as cm
import matplotlib as mpl
from operator import itemgetter
from itertools import groupby
import itertools
import operator
from datetime import timedelta
import csv
import dateutil
import scipy.stats as stats

#Get the data for the verified erupting tracks

In [2]:
#This makes a list of only the track numbers that Ashna verifies as erupting
url_erupting = '/Volumes/Pegasus/nschanch/Laptop/nschanch/REU_2015/Filament_tracking/2012Track_eruptions_text.csv'
track_files = pd.read_csv(url_erupting, header=2)
good_tracks_erupting_2012 = track_files['Track No. '].astype(float)
good_tracks_erupting_2012.head() 


IOError: File /Volumes/Pegasus/nschanch/Laptop/nschanch/REU_2015/Filament_tracking/2012Track_eruptions_text.csv does not exist

In [ ]:
#This is the data file from Dustin with all track information
fi_tracked2012 = pd.read_csv('/Volumes/Pegasus/nschanch/Laptop/nschanch/REU_2015/Filament_tracking/FIDustinTracked_2012.txt',delimiter='\t')   ##working with filament data 
print len(fi_tracked2012)
#fi_tracked2012.head()

In [ ]:
#Only keep tracks with at least 3 data points
good_tracks2012 = fi_tracked2012.groupby('Track No.').filter(lambda x: len(x['Track No.']) >= 3, dropna=True)
print len(good_tracks2012)
good_tracks2012.head()

In [ ]:
 ###selects the data of interest by csv track numbers 
working_data_2012_erupting= good_tracks2012[good_tracks2012['Track No.'].isin(good_tracks_erupting_2012)]
working_data_2012_erupting.head()

In [ ]:
#Pull out the x and y center information
xvals = [loads(elem).x for elem in working_data_2012_erupting['HPC_Center']]
working_data_2012_erupting['xcen'] = xvals
#print len(working_data_2012_trans)
yvals = [loads(elem).y for elem in working_data_2012_erupting['HPC_Center']]
working_data_2012_erupting['ycen'] = yvals 

# Find the Length to width ratio and the area for each filament

In [ ]:
#Find the L to W using the bounding box rectangle.
rec_2012 = working_data_2012_erupting['HPC_BB_Rec']
L_to_W_2012 = []


for item in rec_2012:
    rec = loads(item) #loading each polygon
    #the bounds gives [xmin, ymin, xmax, ymax]
    L_to_W_2012.append((rec.bounds[2]-rec.bounds[0])/ (rec.bounds[3]-rec.bounds[1]))
    
working_data_2012_erupting['L_to_W'] = L_to_W_2012
working_data_2012_erupting.tail()

In [ ]:
poly_2012 = working_data_2012_erupting['HPC_BB_Poly']
area_data_2012 = []


for item in poly_2012:
    poly = loads(item) #loading each polygon
    tot_area = poly.area
    area_data_2012.append(tot_area)
    
working_data_2012_erupting['area'] = area_data_2012

In [ ]:
#Make a new pandas dataframe with the relevant information
all_tracks_erupting = working_data_2012_erupting[['Track No.','fi_length','HEK_time','fi_chirality','fi_tilt','area','fi_barbstot','xcen','ycen','L_to_W']]
print len(all_tracks_erupting)
all_tracks_erupting.head()

In [ ]:
#Just getting to know the data a little bit. 
plt.figure(figsize=(10,10))
plt.grid(True)
plt.ylabel('Number of Erupting Filaments',fontsize=18)
plt.xlabel('Length of Filament (cm)',fontsize=18)
all_tracks_erupting['fi_length'].plot(kind='hist', rwidth=1.5, color = 'orange')
plt.title('Length Distribution of Erupting Filaments',fontsize=18)


    
plt.figure(figsize=(10,10))
plt.grid(True)
plt.ylabel('Number of Erupting Filaments',fontsize=18)
plt.xlabel('Tilt of Filament (Deg)',fontsize=18)
all_tracks_erupting['fi_tilt'].plot(kind='hist', rwidth=1.5, color = 'orange')
plt.title('Tilt Distribution of Erupting Filaments',fontsize=18)

In [ ]:
#Add a column that expresses the time elapsed between filament measurements
grouped_filaments = all_tracks_erupting.groupby('Track No.')
stuff = 0
time_delta = []

for name, group in grouped_filaments:
    #print name
    
    t = group['HEK_time']
    
    time = [pd.to_datetime(date) for date in t]
    t0 = time[0]
    for i in range(len(time)):
        t1 = time[i]
        time_delta.append(timedelta.total_seconds(t1 - t0))
        
#print time_delta       
print len(all_tracks_erupting)
print len(time_delta)
all_tracks_erupting['time_delta'] = time_delta
all_tracks_erupting.head()

In [ ]:
#This will be my base program for plotting the trends for different features

def plot_slopes(x,y,m,b,title,ytitle):
    line = [m*each+b for each in x] #Calculate the line points (y =mx+b)
    
    plt.plot(x, y, color='orange', linestyle='-')
    plt.plot(x, line, '-.', color='blue')   ##plots the line fit
    plt.title(title)
    plt.grid(True)
    fig1 = plt.gcf()
    fig1.set_size_inches(10.5,10.5)
    plt.xlabel('Filament Track time')
    plt.ylabel(ytitle)
    plt.xticks( rotation= 80 )
    #plt.show() #<- uncomment this if you want to plot each filament on a different plot. This takes a lot longer.

# First lets calculate the average LENGTH and the change in length for the filament. 

In [ ]:
grouped_filaments = all_tracks_erupting.groupby('Track No.')
len_slopes = []
avg_len = []
len_stdev = []
len_skew = []
for name, group in grouped_filaments:
    t_delt = group['time_delta']
    f_len = group['fi_length']
    fit = np.polyfit(t_delt, f_len,1)
    len_slopes = len_slopes + [fit]*len(f_len) 
    avg_len.extend([f_len.mean()]*len(f_len))
    len_stdev.extend([np.std(f_len)]*len(f_len))
    len_skew.extend([stats.skew(f_len)]*len(f_len))
    plot_slopes(t_delt,f_len,fit[0],fit[1],'Erupting Filament Length','filament length (cm)')



In [ ]:
len_slopes_only = [item[0] for item in len_slopes]

all_tracks_erupting['len_fits'] = len_slopes
all_tracks_erupting['len_slope'] = len_slopes_only
all_tracks_erupting['avg_len'] = avg_len
all_tracks_erupting['len_stdev'] = len_stdev
all_tracks_erupting['len_skew'] = len_skew
#all_tracks_erupting.head()

In [ ]:
n_groups=5

plt.xlabel('Slope (m) - Length/Time')
plt.ylabel('Number of Instances')
plt.title('Length Change of Erupting Filaments')
fig1 = plt.gcf()
fig1.set_size_inches(8.5,8.5)
plt.hist(len_slopes_only, color='orange')
plt.tight_layout()
plt.show()

#On to the AREA

In [ ]:
#The format of how I find the slope for each feature is the same
grouped_filaments = all_tracks_erupting.groupby('Track No.')
area_slopes = []
avg_area = []
area_stdev = []
area_skew = []
for name, group in grouped_filaments:
    t_delt = group['time_delta']
    f_area = group['area']
    fit = np.polyfit(t_delt, f_area,1)
    area_slopes = area_slopes + [fit]*len(f_area) 
    avg_area.extend([f_area.mean()]*len(f_area))
    area_stdev.extend([np.std(f_area)]*len(f_area))
    area_skew.extend([stats.skew(f_area)]*len(f_area))
    plot_slopes(t_delt,f_area,fit[0],fit[1],'Erupting Filament Area', 'Area (km^2)')

In [ ]:
#Add the area information to the dataframe
area_slopes_only = [item[0] for item in area_slopes]
all_tracks_erupting['avg_area'] = avg_area
all_tracks_erupting['area_fits'] = area_slopes
all_tracks_erupting['area_slope'] = area_slopes_only
all_tracks_erupting['area_stdev'] = area_stdev
all_tracks_erupting['area_skew'] = area_skew
#all_tracks_erupting.head()

In [ ]:
plt.xlabel('Slope (m) - Area/Time')
plt.ylabel('Number of Instances')
plt.title('Change in Area of Erupting Filaments')
fig1 = plt.gcf()
fig1.set_size_inches(8.5,8.5)
plt.hist(area_slopes_only, color='orange')
plt.tight_layout()
plt.show()

#And the Length to Width Ratio

In [ ]:
#The format of how I find the slope for each feature is the same
grouped_filaments = all_tracks_erupting.groupby('Track No.')
size_ratio_slopes = []
avg_size_ratio = []
size_ratio_stdev = []
size_ratio_skew = []
for name, group in grouped_filaments:
    t_delt = group['time_delta']
    f_size_ratio = group['L_to_W']
    fit = np.polyfit(t_delt, f_size_ratio,1)
    size_ratio_slopes = size_ratio_slopes + [fit]*len(f_size_ratio) 
    avg_size_ratio.extend([f_size_ratio.mean()]*len(f_size_ratio))
    size_ratio_stdev.extend([np.std(f_size_ratio)]*len(f_size_ratio))
    size_ratio_skew.extend([stats.skew(f_size_ratio)]*len(f_size_ratio))
    plot_slopes(t_delt,f_size_ratio,fit[0],fit[1],'Erupting Filament L to W', 'L/W')

In [ ]:
#Add the area information to the dataframe
size_ratio_slopes_only = [item[0] for item in size_ratio_slopes]
all_tracks_erupting['avg_L_to_W'] = avg_size_ratio
all_tracks_erupting['L_to_W_slope'] = size_ratio_slopes_only
all_tracks_erupting['L_to_W_stdev'] = size_ratio_stdev
all_tracks_erupting['L_to_W_skew'] = size_ratio_skew
all_tracks_erupting.head()

In [ ]:
plt.xlabel('Size ratio')
plt.ylabel('Number of Instances')
plt.title('Change in shape of Erupting Filaments')
fig1 = plt.gcf()
fig1.set_size_inches(8.5,8.5)
all_tracks_erupting['L_to_W'].plot.hist(color='orange')
#plt.hist(all_tracks_erupting['L_to_W'], color='orange')
#print all_tracks_erupting['L_to_W']
plt.tight_layout()
plt.show()

#Let's check out those barbs

In [ ]:
#barbs = all_tracks_trans['fi_barbstot']
plt.xlabel('Number of Barbs')
plt.ylabel('Number of Instances')
plt.title('Number of barbs in the filament')
fig1 = plt.gcf()
fig1.set_size_inches(8.5,8.5)
#plt.hist(barbs)
plt.tight_layout()
all_tracks_erupting['fi_barbstot'].plot(kind='hist', rwidth=1.5, color = 'orange')

In [ ]:
grouped_filaments = all_tracks_erupting.groupby('Track No.')
avg_barbs = []
barb_slope = []
barb_stdev = []
barb_skew = []
for name, group in grouped_filaments:
    t_delt = group['time_delta']
    barbstot = group['fi_barbstot']
    fit = np.polyfit(t_delt, barbstot,1)
    barb_slope = barb_slope + [fit]*len(barbstot) 
    avg_barbs.extend([barbstot.mean()]*len(barbstot))
    barb_stdev.extend([np.std(barbstot)]*len(barbstot))
    barb_skew.extend([stats.skew(barbstot)]*len(barbstot))
    #plot_slopes(t_delt,barbstot,fit[0],fit[1])
    
barb_slopes_only = [item[0] for item in barb_slope]

In [ ]:
print len(avg_barbs)
print len(barb_slope)
all_tracks_erupting['avg_barbs'] = avg_barbs
all_tracks_erupting['barb_slope'] = barb_slopes_only
all_tracks_erupting['barb_stdev'] = barb_stdev
all_tracks_erupting['barb_skew'] = barb_skew
#all_tracks_erupting.head()

#How tilted is it?

In [ ]:
print min(all_tracks_erupting['fi_tilt'])
print max(all_tracks_erupting['fi_tilt'])

In [ ]:
grouped_filaments = all_tracks_erupting.groupby('Track No.')
tilt_slope = []
avg_tilt = []
tilt_stdev = []
tilt_skew = []
for name, group in grouped_filaments:
    t_delt = group['time_delta']
    f_tilt = group['fi_tilt']
    fit = np.polyfit(t_delt, f_tilt,1)
    tilt_slope = tilt_slope + [fit]*len(f_tilt)  
    avg_tilt.extend([f_tilt.mean()]*len(f_tilt))
    tilt_stdev.extend([np.std(f_tilt)]*len(f_tilt))
    tilt_skew.extend([stats.skew(f_tilt)]*len(f_tilt))
    plot_slopes(t_delt,f_tilt,fit[0],fit[1],'Non-erupting Filament Tilt','Tilt (degrees from equator)')
       
tilt_slopes_only = [item[0] for item in tilt_slope]

In [ ]:
all_tracks_erupting['avg_tilt'] = avg_tilt
all_tracks_erupting['tilt_slope'] = tilt_slopes_only
all_tracks_erupting['tilt_stdev'] = tilt_stdev
all_tracks_erupting['tilt_skew'] = tilt_skew
#all_tracks_erupting.head()

In [ ]:
north_filaments = all_tracks_erupting[all_tracks_erupting['ycen']>=0]
south_filaments = all_tracks_erupting[all_tracks_erupting['ycen']<0]
print len(north_filaments)
print len(south_filaments)
#barbs = all_tracks_erupting['fi_barbstot']
plt.xlabel('Avg tilt')
plt.ylabel('Number of Instances')
plt.xlim([0,90])
plt.title('Absolute value of the Tilt of filaments')
fig1 = plt.gcf()
fig1.set_size_inches(8.5,8.5)
#plt.hist(barbs)
plt.tight_layout()
abs(north_filaments['avg_tilt']).plot(kind='hist', rwidth=1.5, color = 'orange', alpha=0.5)
abs(south_filaments['avg_tilt']).plot(kind='hist', rwidth=1.5, color = 'blue', alpha=0.5)

#Next stop: Chirality

In [ ]:
grouped_filaments = all_tracks_erupting.groupby('Track No.')
chiral_slope = []
avg_chiral = []
for name, group in grouped_filaments:
    t_delt = group['time_delta']
    f_chiral = group['fi_chirality']
    fit = np.polyfit(t_delt, f_chiral,1)
    chiral_slope = chiral_slope + [fit]*len(f_chiral)  
    avg_chiral.extend([f_chiral.mean()]*len(f_chiral))
    plot_slopes(t_delt,f_chiral,fit[0],fit[1],'Erupting Filament Chirality','Chirality (-1 to 1)')
       
chiral_slopes_only = [item[0] for item in chiral_slope]

In [ ]:
all_tracks_erupting['avg_chirality'] = avg_chiral
all_tracks_erupting['chirality_slope'] = chiral_slopes_only
#all_tracks_erupting.head()

#Add the average X and Y coordiantes to the mix for good measure

In [ ]:
grouped_filaments = all_tracks_erupting.groupby('Track No.')
avg_x = []
avg_y = []
for name, group in grouped_filaments:
    f_xloc = group['xcen']
    f_yloc = group['ycen'] 
    avg_x.extend([f_xloc.mean()]*len(f_xloc))
    avg_y.extend([f_yloc.mean()]*len(f_yloc))
all_tracks_erupting['avg_xcen'] = avg_x
all_tracks_erupting['avg_ycen'] = avg_y
all_tracks_erupting.head()

In [ ]:
all_tracks_erupting.keys()

In [ ]:
short_list = grouped_filaments.first()

In [ ]:
short_list.reset_index(inplace=1) #Inplace keeps Track No. as a column, rather than setting it as an index
short_list.head()

In [ ]:
short_list.to_csv('Eruption_Analysis_Updated.csv', columns = ('Track No.', 'avg_len', 'len_slope', 'len_stdev','len_skew','avg_area', 'area_slope', 'avg_chirality','area_stdev','area_skew', 'chirality_slope', 'avg_barbs', 'barb_slope','barb_stdev','barb_skew','avg_tilt', 'tilt_slope','tilt_stdev','tilt_skew','avg_xcen','avg_ycen','avg_L_to_W','L_to_W_slope','L_to_W_stdev','L_to_W_skew'))

In [ ]:
def plot_skew(param_list, color, param):
    plt.xlabel('skew')
    plt.ylabel('Number of Instances')
    plt.xlim([-3,3])
    plt.title('Skew of the %s' % param)
    fig1 = plt.gcf()
    fig1.set_size_inches(5.5,5.5)
    plt.tight_layout()
    param_list.plot(kind='hist',rwidth=1.5, color = color, alpha=0.5)
    plt.show()


plot_skew(short_list['tilt_skew'],color='b', param='tilt')
plot_skew(short_list['area_skew'],color='g', param='area')
plot_skew(short_list['barb_skew'],color='r', param='barb')
plot_skew(short_list['len_skew'],color='y', param='len')
plot_skew(short_list['L_to_W_skew'],color='orange', param='len')

In [ ]:
def plot_stdev(param_list, color, param):
    plt.xlabel('standard deviation')
    plt.ylabel('Number of Instances')
    #plt.xlim([-3,3])
    plt.title('Standard Deviation of the %s' % param)
    fig1 = plt.gcf()
    fig1.set_size_inches(5.5,5.5)
    plt.tight_layout()
    param_list.plot(kind='hist',rwidth=1.5, color = color, alpha=0.5)
    plt.show()


plot_stdev(short_list['tilt_stdev'],color='b', param='tilt')
plot_stdev(short_list['area_stdev'],color='g', param='area')
plot_stdev(short_list['barb_stdev'],color='r', param='barb')
plot_stdev(short_list['len_stdev'],color='y', param='len')
plot_stdev(short_list['L_to_W_stdev'],color='orange', param='len')